In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.models import vgg11
from sklearn.metrics import precision_score, recall_score

/bin/anaconda3/envs/mohamed-chaaben/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/bin/anaconda3/envs/mohamed-chaaben/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
data_path = "/home/mohamed_chaaben/Documents/Colorectal Cancer /"

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

dataset = datasets.ImageFolder(data_path, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

In [4]:
model = vgg11(weights='DEFAULT')

for param in model.features[:-5].parameters():
    param.requires_grad = False

num_classes = 3
model.classifier[6] = nn.Linear(4096, num_classes)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [6]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32)
test_loader = DataLoader(test_set, batch_size=32)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
best_val_loss = float("inf")
patience = 5
no_improvement_count = 0
    
num_epochs = 50
for epoch in range(num_epochs):
    print('start epoch ', epoch)
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {running_loss / len(train_loader)}")

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

        val_loss /= len(val_loader)

    print(f'Validation Loss: {val_loss}')

        # Check for early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= patience:
        print(f'Early stopping after {epoch + 1} epochs with no improvement.')
        break

print('Training finished.')
print('Testing begin.')
model.eval()
correct = 0
total = 0
test_loss = 0.0
    
all_labels = []
all_predicted = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
      
        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
avg_test_loss = test_loss / len(test_loader)
precision = precision_score(all_labels, all_predicted, average='weighted')
recall = recall_score(all_labels, all_predicted, average='weighted')
    
    

print(f'Test Accuracy: {accuracy}%')
print(f'Test Loss: {avg_test_loss}')
print('Test Precision', precision)
print('Test recall', recall)

start epoch  0
Epoch 1/50 - Loss: 0.36162110830346744
Validation Loss: 0.19544666731043867
start epoch  1
Epoch 2/50 - Loss: 0.16920048875113328
Validation Loss: 0.1299336797704822
start epoch  2
Epoch 3/50 - Loss: 0.11020502803536752
Validation Loss: 0.10498504139679044
start epoch  3
Epoch 4/50 - Loss: 0.0760997954911242
Validation Loss: 0.10884721283065646
start epoch  4
Epoch 5/50 - Loss: 0.05211911250837147
Validation Loss: 0.09526251952506994
start epoch  5
Epoch 6/50 - Loss: 0.03673171467380598
Validation Loss: 0.08713774005637358
start epoch  6
Epoch 7/50 - Loss: 0.026448198912354805
Validation Loss: 0.08569073784900338
start epoch  7
Epoch 8/50 - Loss: 0.019103585136084195
Validation Loss: 0.09042248354097338
start epoch  8
Epoch 9/50 - Loss: 0.013228973911221449
Validation Loss: 0.09329451865663654
start epoch  9
Epoch 10/50 - Loss: 0.011132665918557905
Validation Loss: 0.09361850097775459
start epoch  10
Epoch 11/50 - Loss: 0.010058770772878537
Validation Loss: 0.09181193305

In [ ]:
#save this model to use it later for tsne